In [2]:
import random
import pandas as pd
import sklearn
import matplotlib.pyplot as plt 
import numpy as np
import seaborn
import json
import pickle
from pandas.io.json import json_normalize
from nltk.corpus import stopwords
import re
import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone

In [3]:
with open("../bow_data.json") as file:
    json_ext = json.load(file)

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [9]:
from sklearn import svm
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn_hierarchical_classification.constants import ROOT
from sklearn_hierarchical_classification.metrics import h_fbeta_score, multi_labeled
from sklearn_hierarchical_classification.tests.fixtures import make_digits_dataset

In [61]:
# Used for seeding random state
RANDOM_STATE = 42


def classify_digits():
    r"""Test that a nontrivial hierarchy leaf classification behaves as expected.

    We build the following class hierarchy along with data from the handwritten digits dataset:

            <ROOT>
           /      \
          A        B
         / \       |  \
        1   7      C   9
                 / | \
                3  4  8

    """
    class_hierarchy = {
        ROOT: ["A", "B"],
        "A": ["1", "7"],
        "B": ["C", "9"],
        "C": ["3", "4", "8"],
    }
    base_estimator = make_pipeline(
        TruncatedSVD(n_components=24),
        svm.SVC(
            gamma=0.001,
            kernel="rbf",
            probability=True
        ),
    )
    clf = HierarchicalClassifier(
        base_estimator=base_estimator,
        class_hierarchy=class_hierarchy,
    )
    X, y = make_digits_dataset(
        targets=[1, 7, 3, 4, 8, 9],
        as_str=False,
    )
    print(y.shape)
    # cast the targets to strings so we have consistent typing of labels across hierarchy
    y = y.astype(str)

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=RANDOM_STATE,
    )

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print("Classification Report:\n", classification_report(y_test, y_pred))

    # Demonstrate using our hierarchical metrics module with MLB wrapper
    with multi_labeled(y_test, y_pred, clf.graph_) as (y_test_, y_pred_, graph_):
        h_fbeta = h_fbeta_score(
            y_test_,
            y_pred_,
            graph_,
        )
        print("h_fbeta_score: ", h_fbeta)


if __name__ == "__main__":
    classify_digits()

(1079,)
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00        35
           3       1.00      1.00      1.00        30
           4       1.00      1.00      1.00        40
           7       1.00      1.00      1.00        37
           8       0.91      1.00      0.96        32
           9       1.00      0.93      0.96        42

    accuracy                           0.99       216
   macro avg       0.99      0.99      0.99       216
weighted avg       0.99      0.99      0.99       216

h_fbeta_score:  0.9915966386554622


In [17]:
with open("../../category_article.json") as file:
    json_ext = json.load(file)

In [18]:
flat = {}
def flatten_hierarchy(json_obj):
    flat[json_obj["name"]] = [obj["name"] for obj in json_obj["children"]]
    for i in json_obj["children"]:
        flatten_hierarchy(i)

In [23]:
flatten_hierarchy(json_ext)

In [24]:
flat

{'Root': ['Hobbies and Crafts',
  'Sports and Fitness',
  'Relationships',
  'Pets and Animals',
  'Finance and Business',
  'Computers and Electronics',
  'Cars & Other Vehicles',
  'Education and Communications',
  'Home and Garden',
  'Personal Care and Style',
  'Health',
  'Philosophy and Religion',
  'Youth',
  'Family Life',
  'Arts and Entertainment',
  'Work World',
  'Holidays and Traditions',
  'Food and Entertaining',
  'Travel',
  'Singapore',
  'Canada',
  'United Kingdom',
  'Outdoor Shelters',
  'Art Therapy',
  'Paper Roses',
  'Screenplays',
  'Cleaning Heater Appliances'],
 'Hobbies and Crafts': ['Games',
  'Photography',
  'Digital Technology Art and Craft Projects',
  'Crafts',
  'Collecting',
  'Rocks and Minerals',
  'Woodworking',
  'Amateur Radio',
  'Toys',
  'Drawing',
  'Model Making',
  'Kite Making and Kite Flying',
  'Optical Devices',
  'Flags'],
 'Games': ['Video Games',
  'Backyard Games',
  'Games for Children',
  'Card Games',
  'Spying',
  'Wargamin

In [25]:
for k, v in flat.items():
    flat[k].append(k + " - Container")

In [26]:
flat

{'Root': ['Hobbies and Crafts',
  'Sports and Fitness',
  'Relationships',
  'Pets and Animals',
  'Finance and Business',
  'Computers and Electronics',
  'Cars & Other Vehicles',
  'Education and Communications',
  'Home and Garden',
  'Personal Care and Style',
  'Health',
  'Philosophy and Religion',
  'Youth',
  'Family Life',
  'Arts and Entertainment',
  'Work World',
  'Holidays and Traditions',
  'Food and Entertaining',
  'Travel',
  'Singapore',
  'Canada',
  'United Kingdom',
  'Outdoor Shelters',
  'Art Therapy',
  'Paper Roses',
  'Screenplays',
  'Cleaning Heater Appliances',
  'Root - Container'],
 'Hobbies and Crafts': ['Games',
  'Photography',
  'Digital Technology Art and Craft Projects',
  'Crafts',
  'Collecting',
  'Rocks and Minerals',
  'Woodworking',
  'Amateur Radio',
  'Toys',
  'Drawing',
  'Model Making',
  'Kite Making and Kite Flying',
  'Optical Devices',
  'Flags',
  'Hobbies and Crafts - Container'],
 'Games': ['Video Games',
  'Backyard Games',
  'Ga

In [27]:
with open("../bow_data.json") as file:
    json_ext = json.load(file)

In [80]:
total_bow = []
def collect_bow(json_obj, path=[]):
    path.append(json_obj["name"])
    
    for i in json_obj["children"]:
        collect_bow(i, path[:])
    
    for j in json_obj["content"]:
#         if "meta" in j:
        total_bow.append([j["name"].lower(), json_obj["name"] + " - Container"])

collect_bow(json_ext)

In [63]:
df = pd.DataFrame(total_bow)

In [64]:
df.head()

,0,1
0,[be good at halo 2 without being cheap],Halo 2 - Container
1,[dual wield in halo 2],Halo 2 - Container
2,[get halo 2 for your pc],Halo 2 - Container
3,[get the scarab gun in halo 2],Halo 2 - Container
4,[kill the brute at the end of halo 2],Halo 2 - Container


In [69]:
flat["<ROOT>"] = flat["Root"]

In [70]:
del flat["Root"]

In [85]:
X = np.asarray([[entry[0]] for entry in total_bow])
y = np.asarray([entry[1] for entry in total_bow])
X.astype(str)

array([['be good at halo 2 without being cheap'],
       ['dual wield in halo 2'],
       ['get halo 2 for your pc'],
       ...,
       ['vote on featured article candidates on wikihow'],
       ['wake up cyrus in animal crossing: new leaf'],
       ['write a good edit summary on wikihow']], dtype='<U127')

In [74]:
# Used for seeding random state
RANDOM_STATE = 42


def classify_docs():
    r"""Test that a nontrivial hierarchy leaf classification behaves as expected.

    We build the following class hierarchy along with data from the handwritten digits dataset:

            <ROOT>
           /      \
          A        B
         / \       |  \
        1   7      C   9
                 / | \
                3  4  8

    """
    base_estimator = make_pipeline(CountVectorizer(),
                                   TfidfVectorizer(),
                                   SGDClassifier(
                                       loss='modified_huber',
                                       penalty='l2',
                                       alpha=1e-3,
                                       random_state=42,
                                       max_iter=5,
                                       tol=None)
                                  )
#     base_estimator = make_pipeline(
#         TruncatedSVD(n_components=24),
#         svm.SVC(
#             gamma=0.001,
#             kernel="rbf",
#             probability=True
#         ),
#     )
    clf = HierarchicalClassifier(
        base_estimator=base_estimator,
        class_hierarchy=flat,
    )
#     X, y = make_digits_dataset(
#         targets=[1, 7, 3, 4, 8, 9],
#         as_str=False,
#     )
    X = np.asarray([entry[0] for entry in total_bow])
    y = np.asarray([entry[1] for entry in total_bow])
    # cast the targets to strings so we have consistent typing of labels across hierarchy
#     y = y.astype(str)

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=RANDOM_STATE,
    )

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print("Classification Report:\n", classification_report(y_test, y_pred))

    # Demonstrate using our hierarchical metrics module with MLB wrapper
    with multi_labeled(y_test, y_pred, clf.graph_) as (y_test_, y_pred_, graph_):
        h_fbeta = h_fbeta_score(
            y_test_,
            y_pred_,
            graph_,
        )
        print("h_fbeta_score: ", h_fbeta)

classify_docs()

c:\users\t-sesan\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:532: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)


TypeError: no supported conversion for types: (dtype('<U127'),)